## Classes and Data Models

### Data_Loading Class

In [1]:
import pandas as pd

class Data_Loading:
    def __init__(self, csv_path):
        self.df = pd.read_csv(csv_path, dtype={'ZIP':str, 'MSA':str})
                                        
    

### Mapping Class

In [2]:
class Mapping:
    def __init__(self, df):
        self.df = df

    def get_states(self):
        # list of extraced state codes 
        states = sorted(self.df['State'].unique().tolist())

        # dictionary of mapped numbers and states
        state_dict = {str(i): s for i, s in enumerate(states, start=1)}

        return state_dict

    def get_metros(self, input_state):
        
        # trimmed dataframe of state, metro pairs, dropped duplicate pairs
        metro_state = (self.df[['State', 'Metro']].drop_duplicates())

        # dictionary of state:metros tuples               
        metro_state_dict = {
            state: sorted(group['Metro'].tolist())
            for state, group in metro_state.groupby('State')}

        # list of metro areas in selected state
        metros = metro_state_dict[input_state]

        # dictionary of mapped numbers and metro areas in selected state
        metros_dict = {str(i): m for i, m in enumerate(metros, start=1)}

        return metros_dict

    def get_profiles(self):
        # trimmed dataframe to remove duplicate ZIP/cluster values that were exploded for metro areas that span multiple states
        unique_ZIPs_df = self.df[['ZIP', 'cluster', 'Avg.HeatIndex.Mean']].drop_duplicates()

        # creates cluster profiles from trimmed ZIP/cluster dataframe
        cluster_profiles = (
            unique_ZIPs_df
            .groupby('cluster')['Avg.HeatIndex.Mean']
            .agg(High='max', Low='min', Avg='mean')
            .reset_index()
        )

        # creates cluster dictionary with number mapping for menu use
        profile_dict = {}
        for _, row in cluster_profiles.iterrows():
            key = str(int(row.cluster) + 1)
            profile_data = (
                f"Cluster: {int(row.cluster)}, "
                f"High: {row.High:.1f}, "
                f"Low: {row.Low:.1f}, "
                f"Avg: {row.Avg:.1f}"
            )
            profile_dict[key] = profile_data

        return profile_dict

    def get_matching_metros(self, selected_metro_name):
        cluster_ids_set = set(self.df.loc[self.df['Metro'] == selected_metro_name, 'cluster'].tolist())
        matching_metros = []
        for i in sorted(cluster_ids_set):
            key = str(i + 1)
            profile = self.get_profiles()[key]
            matches = sorted(
                self.df.loc[self.df['cluster'] == i, 'Metro']
                .unique()
                .tolist()
            )
            matches.remove(selected_metro_name)
            matching_metros.append((key, profile, matches))

        return matching_metros

    def get_matching_profiles(self, selected_profile: int):
        matching_profile_metros = sorted(
            self.df.loc[self.df['cluster'] == selected_profile, 'Metro']
            .unique()
            .tolist()
        )
        return matching_profile_metros
     

### Menu Class

In [3]:
class Menu:
    def __init__(self):
        pass

    # main menu logic, including escape
    # returns user chosen match option
    def main_menu(self):
        print("Welcome to the Climate-Match System! \nSelect a number to begin: \n")
        choice_main = input("1. Match Climate by State and Metropolitan Area.\n2. Match Climate by Profile\nQ to Quit")
    
        return choice_main

    # state menu logic, allows enumeration from dict to be used as menu option, includes escape
    # returns user chosen state 
    def state_menu(self, state_dict):
        state_menu = []
        for key in sorted(state_dict.keys(), key=int):
            state_menu.append(f"{key}. {state_dict[key]}")
        state_menu.append("Q to Quit")
    
        choice_state = input("\n".join(state_menu))
    
        return choice_state

    # metro menu logic, allows enumeration from dict to be used as menu option, includes escape
    # returns user chosen metro area
    def metro_menu(self, metro_dict):
        metro_menu = []
        for key in sorted(metro_dict.keys(), key=int):
            metro_menu.append(f"{key}. {metro_dict[key]}")
        metro_menu.append("Q to Quit")
        metro_menu.append("Select a metropolitan area by number: ")
        
        choice_metro = input("\n".join(metro_menu))
    
        return choice_metro

    # climate profile menu logic, allows key from dict to be used as menu option, includes escape
    # returns user chosen climate profile
    def profile_menu(self, profile_dict):
        profile_menu = []
        for key in sorted(profile_dict.keys(), key=int):
            profile_menu.append(f"{key}. {profile_dict[key]}")
        profile_menu.append("Q to Quit")
        profile_menu.append("Select a climate profile by number: ")
    
        choice_profile = input("\n".join(profile_menu))
    
        return choice_profile
        
    

## Main Controller

In [ ]:
import os
import pandas as pd

# instantiate class objects
dl = Data_Loading('clustered_ZIP_climates.csv')
mapping = Mapping(dl.df)
menu = Menu()

# menu loop
while True:
    # main menu
    choice_main = menu.main_menu()

    if choice_main in ('q', 'Q'):
        print("Goodbye!")
        break

    else:
        print(choice_main)
        
        while True:
            match choice_main:
                case '1':
                    # state menu
                    choice_state = menu.state_menu(mapping.get_states())
    
                    if choice_state in ('q', 'Q'):
                        print("Goodbye!")
                        break
    
                    elif choice_state in mapping.get_states():    
                        print(choice_state)
                        
            
                        while True:
                            match choice_state:
                                case x if x in mapping.get_states().keys():
                                    input_state = mapping.get_states()[x]
                                    metros_dict = mapping.get_metros(input_state)

                                    # metro menu
                                    choice_metro = menu.metro_menu(metros_dict)
        
                                    if choice_metro in ('q', 'Q'):
                                        print("Goodbye!")
                                        break
                                        
                                    elif choice_metro in metros_dict:    
                                        selected_metro_name = metros_dict[choice_metro]
                                        print("Please note some metropolitan areas include ZIP codes in multiple states.\n")
                                        print(choice_metro)
                                        
                                        while True:
                                            match choice_metro:    
                                                case 'q'|'Q':
                                                    print("Goodbye!")
                                                    break
                
                                                case x if x in metros_dict.keys():
                                                    selected_metro = metros_dict[x]
                                                    matching_metros = mapping.get_matching_metros(selected_metro_name) 
                                                    for key, profile, matches in matching_metros:
                                                        print("Please note some metropolitan areas include ZIP codes in multiple clusters.")
                                                        print("All matching clusters are presented for most complete information.")
                                                        print(f"\n{selected_metro_name}")
                                                        print("_______________________")
                                                        print(f"{profile}")
                                                        print("_______________________")
                                                        print("\n".join(matches))
                                                        print("\n\n")    
                                                    break
    
                                                case _:
                                                    print("Invalid input, please try again.")
                                                    break
                                    else:
                                        print("Invalid input, please try again.") 
    
                                case 'q'|'Q':
                                    print("Goodbye!")
                                    break
            
                                case _:
                                    print("Invalid input, please try again.")
                                    break
    
                            
                    else:
                        print("Invalid input, please try again.")
                                                
                                    
       
                case '2':
                    # climate profile menu
                    profile_dict = mapping.get_profiles()
                    choice_profile = menu.profile_menu(profile_dict)

                    if choice_profile in ('q', 'Q'):
                        break

                    else:
                        while True:
                            match choice_profile:
                                case x if x in profile_dict.keys():
                                    selected_profile = int(x) - 1
                                    matching_profile_metros = mapping.get_matching_profiles(selected_profile) 
                                    print("Please note some metropolitan areas include ZIP codes in multiple clusters.")
                                    print("All matching clusters are presented for most complete information.")
                                    print("_______________________")
                                    print(profile_dict[choice_profile])
                                    print("_______________________")
                                    for matches in matching_profile_metros:
                                        print(matches)
                                    print("\n\n")      
                                    break 
    
                                case 'q'|'Q':
                                    print("Goodbye!")
                                    break
    
                                case _:
                                    print("Invalid input, please try again.")
                                    break  
                        
                        
            
                case 'q'|'Q':
                    print("Goodbye!")
                    break
        
                case _:
                    print("Invalid input, please try again.")
                    break


Welcome to the Climate-Match System! 
Select a number to begin: 



1. Match Climate by State and Metropolitan Area.
2. Match Climate by Profile
Q to Quit 1


1


1. AL
2. AR
3. AZ
4. CA
5. CO
6. CT
7. DC
8. DE
9. FL
10. GA
11. IA
12. ID
13. IL
14. IN
15. KS
16. KY
17. LA
18. MA
19. MD
20. ME
21. MI
22. MN
23. MO
24. MS
25. NC
26. NE
27. NJ
28. NM
29. NV
30. NY
31. OH
32. OK
33. OR
34. PA
35. RI
36. SC
37. TN
38. TX
39. UT
40. VA
41. WA
42. WI
43. WV
Q to Quit 25


25


1. Charlotte-Concord-Gastonia
2. Durham-Chapel Hill
3. Greensboro-High Point
4. Raleigh
5. Virginia Beach-Norfolk-Newport News
6. Winston-Salem
Q to Quit
Select a metropolitan area by number:  1


Please note some metropolitan areas include ZIP codes in multiple states.

1
Please note some metropolitan areas include ZIP codes in multiple clusters.
All matching clusters are presented for most complete information.

Charlotte-Concord-Gastonia
_______________________
Cluster: 4, High: 83.3, Low: 28.5, Avg: 56.4
_______________________
Albuquerque
Baltimore-Columbia-Towson
Camden
Cincinnati
Kansas City
Knoxville
Louisville/Jefferson County
Montgomery County-Bucks County-Chester County
Nashville-Davidson--Murfreesboro--Franklin
Philadelphia
Richmond
Silver Spring-Frederick-Rockville
St. Louis
Washington-Arlington-Alexandria
Wichita
Wilmington
Winston-Salem



Please note some metropolitan areas include ZIP codes in multiple clusters.
All matching clusters are presented for most complete information.

Charlotte-Concord-Gastonia
_______________________
Cluster: 8, High: 89.0, Low: 35.7, Avg: 61.6
_______________________
Atlanta-Sandy Springs-Roswell
Augusta-Richmond County
Birmingham-H